### App
To access the Spotify API first you need to create a App by going to https://developer.spotify.com/dashboard.

| | |
|--- | --- |
| App name | Fabric |
| App description | Test |
| Website | https://github.com/jakeduddy |
| Redirect UI | http://localhost:8888/callback |

The app is given a `client_id` and `client_secret`.

In [ ]:
import os
os.environ['CLIENT_ID'] = '****'
os.environ['CLIENT_SECRET'] = '****'

### Authorization

There are several OAuth flows that can be used.

| FLOW	| Access User Resources | Requires Secret Key (Server-Side) | Access Token Refresh |
| --- | --- | --- | --- |
| Authorization code | Yes | Yes | Yes |
| Authorization code with PKCE | Yes | No | Yes |
| Client credentials | No | Yes | No |
| Implicit grant | Yes | No | No |

To access user information the Authorization code flow will be used.

![image-alt-text](https://developer.spotify.com/images/documentation/web-api/auth-code-flow.png)

#### 1 Request Authorization To Access Data
A user must perform a one-time authorization of scope, by following a url and interating with a web dialog. The reponse returns a callback with a code.

Scopes: https://developer.spotify.com/documentation/web-api/concepts/scopes

In [ ]:
def get_authorization_url(scope):
    client_id = os.environ['CLIENT_ID']
    query = f'?client_id={client_id}&response_type=code&redirect_uri=http://localhost:8888/callback&scope={scope}'
    url = f'https://accounts.spotify.com/authorize{query}'
    print(url)

get_authorization_url(scope='user-read-currently-playing%20user-top-read')

In [ ]:
os.environ["AUTHORIZATION_CODE"] = '****'

#### 2 Request Access and Refresh Tokens
If the user accepted your request, then your app is ready to exchange the authorization code for an Access Token. It can do this by making a POST request to the /api/token endpoint.

The `access_token` can be used for subsequent API calls. When the `access_token` expires the `refresh_token` can be sent to the Spotify Accounts service in place of an `authorization_code` to get a new `access_token`.

In [ ]:
import requests
import base64
import json

def get_access_token():
    url= 'https://accounts.spotify.com/api/token'
    client_id = os.environ['CLIENT_ID']
    client_secret = os.environ['CLIENT_SECRET']
    auth_string = f'{client_id}:{client_secret}'
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    headers = {
        'Authorization': f'Basic {auth_base64}',
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    data= {
        'grant_type': 'authorization_code',
        'code': os.environ['AUTHORIZATION_CODE'],
        'redirect_uri': 'http://localhost:8888/callback'
    }
    response = requests.post(url, headers=headers, data=data)
    print(response)
    response_json = response.json()
    print(response.json())
    os.environ['ACCESS_TOKEN'] = response_json['access_token']
    os.environ['AUTHORIZATION_CODE'] = response_json['refresh_token']

get_access_token()

### 3 Use access token in request to Web API
Request Top Tracks

In [ ]:
def get_users_top_tracks(time_range, limit, offset):
    query = f'?time_range={time_range}?limit={limit}?offset={offset}'
    # url = f'https://api.spotify.com/v1/me/top/tracks{query}'
    url = 'https://api.spotify.com/v1/me/top/tracks'
    access_token = os.environ['ACCESS_TOKEN']
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)
    return response

In [ ]:
import pandas as pd

top_tracks = get_users_top_tracks(time_range='medium_term', limit=1, offset=0).json()
df = pd.DataFrame.from_dict(top_tracks)

df.to_json('abfss://73929f9f-e2db-4c4d-b069-9772a2af25d7@onelake.dfs.fabric.microsoft.com/37ad89b4-2cc3-4fea-a904-c9cd897d2b9d/Files/Top_Track/Top_Track.json')